**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 3.4MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 


Read the RDF file

In [3]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=Na3eb93ac69c144588e1ee7ab556b228e (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [7]:
ns = Namespace("http://somewhere#")
for s, p, o in g.triples((None, RDFS.subClassOf, ns.Person)):
  print(s)
  for s1, p1, o1 in g.triples((None, RDFS.subClassOf, s)):
    print(s1)

http://somewhere#Researcher
http://somewhere#PhDstudent


In [12]:
from rdflib.plugins.sparql import prepareQuery

q1 = prepareQuery("""
  SELECT DISTINCT ?x 
  WHERE 
  {
	?x rdfs:subClassOf* ns:Person.
  }
  """,
  initNs = { "ns": ns}
)
for x in g.query(q1):
  print(x)

(rdflib.term.URIRef('http://somewhere#Person'),)
(rdflib.term.URIRef('http://somewhere#Researcher'),)
(rdflib.term.URIRef('http://somewhere#PhDstudent'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [11]:
for s, p, o in g.triples((None, RDF.type, ns.Person)):
  print(s)
for s, p, o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s1,p1,o1 in g.triples((None, RDF.type, s)):
    print(s1)

http://somewhere#JohnSmith
http://somewhere#SaraJones
http://somewhere#JaneSmith


In [14]:
q2 = prepareQuery("""
  SELECT DISTINCT ?x  
  WHERE {
  ?x a/rdfs:subClassOf* ns:Person.
  }""",
initNs = {"ns":ns}
)
for x in g.query(q2):
  print(x)

(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [15]:
for s, p, o in g.triples((None, RDF.type, ns.Person)):
  for s1, p1, o1 in g.triples((s, None, None)):
    print(s1, p1, o1)

for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s1, p1, o1 in g.triples((None, RDFS.subClassOf, s)):
    for s2, p2, o2 in g.triples((s1, None, None)):
      print(s2, p2, o2)


http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#PhDstudent http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Class
http://somewhere#PhDstudent http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Researcher


In [16]:
q3 = prepareQuery("""
  SELECT ?z ?x ?y
  WHERE {
      ?s rdfs:subClassOf/rdfs:subClassOf* ns:Person.
      ?z rdf:type ?s.
      ?z ?x ?y
  }
  """,
  initNs = {"ns":ns}
)

for x in g.query(q3):
  print(x)

(rdflib.term.URIRef('http://somewhere#JaneSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given'), rdflib.term.Literal('Jane', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#JaneSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family'), rdflib.term.Literal('Smith', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#JaneSmith'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://somewhere#Researcher'))
(rdflib.term.URIRef('http://somewhere#JaneSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN'), rdflib.term.Literal('jane Smith', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
